In [33]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [34]:
def match_and_join_link(link):
    data = []
    file_txt = os.listdir(link)
    for file in file_txt:
        link_to_file_txt = os.path.join(link,file)
        data.append(link_to_file_txt)
    return data

In [35]:
def read_data_txt(link_txt_list):
    txt_list = []
    n = 50
    for link in link_txt_list:
#         txt = os.open(link,os.O_RDWR)
#         txt = os.read(link,n)
        txt = open(link,'r')
        txt = txt.read()
        txt_list.append(txt)
    return txt_list

## Train

In [36]:
link_positive_file = '/sentiment_data/data_train/train/pos'
link_negative_file = '/sentiment_data/data_train/train/neg'

In [37]:
link_positive_data = match_and_join_link(link_positive_file)
link_negative_data = match_and_join_link(link_negative_file)


In [38]:
positive_data_list = read_data_txt(link_positive_data)
negative_data_list = read_data_txt(link_negative_data)
X_data_traning = positive_data_list.copy()
X_data_traning.extend(negative_data_list)
X_data_traning = np.array(X_data_traning)
X_data_traning.shape

(30000,)

In [39]:
#create tager
positive_label = [1]*len(positive_data_list)

negative_label = [0]*len(negative_data_list)
labels_train = positive_label.copy()
labels_train.extend(negative_label) 
labels_train = np.array(labels)
labels_train.shape

(30000,)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
X_train_counts = count_vector.fit_transform(X_data_traning)
X_train_counts.shape


(30000, 35464)

## Valid

In [41]:
link_positive_valid_file = '/home/sangdh/Downloads/text_classification/sentiment_data/data_train/valid/pos'
link_negative_valid_file = '/home/sangdh/Downloads/text_classification/sentiment_data/data_train/valid/neg'

In [42]:
link_positive_valid_data = match_and_join_link(link_positive_valid_file)
link_negative_valid_data = match_and_join_link(link_negative_valid_file)

In [43]:
positive_data_valid_list = read_data_txt(link_positive_valid_data)
negative_data_valid_list = read_data_txt(link_negative_valid_data)

In [44]:
X_data_valid = positive_data_valid_list.copy()
X_data_valid.extend(negative_data_valid_list)
X_data_valid = np.array(X_data_valid)
X_data_valid.shape


(10000,)

In [45]:
#create target valid
labels_positive_valid = [1]*len(positive_data_valid_list)
labels_negative_valid = [0]*len(negative_data_valid_list)
labels_valid = labels_positive_valid.copy()
labels_valid.extend(labels_negative_valid)
labels_valid = np.array(labels_valid)
labels_valid.shape

(10000,)

# TF-IDF

## TF-IDF

In [46]:
from sklearn.feature_extraction.text import TfidfTransformer

In [47]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(30000, 35464)

In [48]:
#count vector: valid
# count_vector = CountVectorizer()
X_valid_counts = count_vector.transform(X_data_valid)
X_valid_counts.shape

#tf-idf valid
# tfidf_transformer = TfidfTransformer()
X_valid_tfidf = tfidf_transformer.transform(X_valid_counts)
X_valid_tfidf.shape


(10000, 35464)

## model_SGDclassifier

### training data

In [49]:
from sklearn.linear_model import SGDClassifier
model_SDG_classifier = SGDClassifier()
model_SDG_classifier.fit(X_train_tfidf,labels_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

### prediction

In [53]:
train_predicted = model_SGD_classifier.predict(X_train_tfidf)
valid_predicted = model_SGD_classifier.predict(X_valid_tfidf)

### evaluation

In [51]:
from sklearn import metrics

train_evaluation

In [54]:
print(metrics.classification_report(labels_train,train_predicted))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91     15000
           1       0.90      0.93      0.91     15000

    accuracy                           0.91     30000
   macro avg       0.91      0.91      0.91     30000
weighted avg       0.91      0.91      0.91     30000



valid_ evaluation

In [55]:
print(metrics.classification_report(labels_valid,predicted))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      5000
           1       0.88      0.90      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [56]:
metrics.confusion_matrix(labels_valid,predicted)

array([[4361,  639],
       [ 525, 4475]])

## model naive_bayes

### training

In [60]:
from sklearn.naive_bayes import MultinomialNB
model_naive_bayes = MultinomialNB()
model_naive_bayes.fit(X_train_tfidf,labels_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### prediction

In [61]:
train_naive_bayes_predicted = model_naive_bayes.predict(X_train_tfidf)
valid_naive_bayes_predicted = model_naive_bayes.predict(X_valid_tfidf)

### evaluation

train_evaluation

In [62]:
print(metrics.classification_report(labels_train,train_naive_bayes_predicted))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88     15000
           1       0.88      0.89      0.88     15000

    accuracy                           0.88     30000
   macro avg       0.88      0.88      0.88     30000
weighted avg       0.88      0.88      0.88     30000



valid_evaluation

In [66]:
print(metrics.classification_report(labels_valid,valid_naive_bayes_predicted))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      5000
           1       0.86      0.85      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



## Random forest model

### training

In [79]:
from sklearn.ensemble import RandomForestClassifier

In [80]:
model_random_forest = RandomForestClassifier()
model_random_forest.fit(X_train_tfidf,labels_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### prediction

predicted

In [90]:
train_random_forest_prediction = model_random_forest.predict(X_train_tfidf)
valid_random_forest_prediction = model_random_forest.predict(X_valid_tfidf)
train_random_forest_prediction.shape

(30000,)

predicted_proba

In [82]:
train_random_forest_prediction = model_random_forest.predict_proba(X_train_tfidf)
valid_random_forest_prediction = model_random_forest.predict_proba(X_valid_tfidf)

### evaluation

train_evaluation

In [99]:
print(model_random_forest.score(X_train_tfidf,train_random_forest_prediction))

1.0


valid_evaluation

In [102]:
print(model_random_forest.score(X_valid_tfidf,valid_random_forest_prediction))

1.0


## logistic_model


### training

In [93]:
from sklearn.linear_model import LogisticRegression

In [95]:
model_logistic = LogisticRegression()
model_logistic.fit(X_train_tfidf, labels_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### prediction

In [96]:
train_logistic_predicted = model_logistic.predict(X_train_tfidf)
valid_logistic_predicted = model_logistic.predict(X_valid_tfidf)

### evaluation

train_evaluation

In [97]:
print(model_logistic.score(X_train_tfidf,labels_train))

0.9134333333333333


valid_evaluation

In [98]:
print(model_logistic.score(X_valid_tfidf,labels_valid))

0.8837


# Hasing